In [ ]:
import pandas as pd
import os
os.getcwd()
os.chdir('/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/liujincheng06/task2')
#from call import generate_answer_useGpt4

In [ ]:
from call import generate_answer_useGpt4

In [ ]:
df = pd.read_excel("mbpp_cpp.xlsx")

In [ ]:
codes = df['code_str']

In [ ]:
assert_stmts = df['test']

In [ ]:
codes_with_test_assert = []
for i in range(len(codes)):
    codes_with_test_assert.append(codes[i] + "\n" + assert_stmts[i])

In [ ]:
print(codes_with_test_assert[0])

In [ ]:
cpp_transferd_codes = []
cpp_transferd_tests = []
cpp_back = []

In [ ]:
import re

def extract_cpp_sections(cpp_code):
    # 匹配 main 函数部分
    main_pattern = re.compile(r'int\s+main\s*\([^)]*\)\s*\{.+\}', re.DOTALL)
    main_function = main_pattern.search(cpp_code)
    main_function = main_function.group() if main_function else ""

    code_str = main_pattern.sub("",cpp_code)
    code_str = re.sub("^(`+).*","",code_str)
    code_str = re.sub("```","",code_str)

    return code_str,main_function

In [ ]:
from tqdm import tqdm

In [ ]:
import json

with open("mbpp_cpp_tests.jsonl","w") as f:
    for i,v in tqdm(enumerate(codes_with_test_assert)):
        if df['is_passed'][i]:
            continue
        prompt = """你的任务是进行代码翻译，源代码语言为python，翻译后代码语言为c++。
    代码中含有一个函数，一些测试用例，可以直接运行。
    要求翻译后的代码也能直接运行。我会给你待翻译代码的一个翻译结果，但它是错误的，你可以结合错误信息继续翻译。
    待翻译的python代码内容:
    %s
    这是该python代码的错误的c++翻译结果和编译器给出的错误信息，你可以在它的基础上改，也可以自己重新翻译:
    %s
    错误信息:
    %s
    要求:
    1. 只需要给出翻译后的代码，不需要文字解释；
    2. 请按照指定格式给出代码
    3. 注意include的包完整，一定要有cassert这个include
    4. 测试用例一定要使用assert，不要使用cout的形式

    格式:
    ```cpp
    //include列表
    using namespace std;
    //函数定义
    int main()
    {
        //使用assert的测试用例
    }
    ```
                            
                            """ %(v,df['cpp_code_str'][i] + df['cpp_test'][i],df['unpassed_error_info'][i])
        generation = generate_answer_useGpt4(prompt)
        cpp_code_str,cpp_test = extract_cpp_sections(generation)
        f.write(json.dumps({'cpp_code_str':cpp_code_str,'cpp_test':cpp_test,'backup':generation}) + "\n")
        cpp_transferd_codes.append(cpp_code_str)
        cpp_transferd_tests.append(cpp_test)
        cpp_back.append(generation)
        

        

In [ ]:
df2 = pd.DataFrame(columns=['cpp_code_str','cpp_test','backup'])

In [ ]:
df2['cpp_code_str'] = cpp_transferd_codes
df2['cpp_test'] = cpp_transferd_tests
df2['backup'] = cpp_back

In [ ]:
df2.to_excel('mbpp_rest.xlsx')